In [1]:
import pandas as pd
import pyodbc
from datetime import datetime

In [2]:
sql_query = """
    -- all renewal WO and their flocid
    SELECT
        [FunctionLocation],
        CASE
            WHEN [TechCompletionDate] IS NULL THEN [BasicFinishDate]
            ELSE [TechCompletionDate]
        END AS 'CompletionDate'
    FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.ringfenced].[vw_WorkOrder]
    WHERE
        OrderType = 'MW04'
        AND CompanyCode = '5000'
        AND MainUserStatusDesc = 'Practically Completed'
        AND ActualTotalCost IS NOT NULL
        AND ActualTotalCost > 0
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';DATABASE=myANALYTICS_SP;Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_renewal_wo = pd.read_sql_query(sql_query, conn)

# Display the DataFrame
all_renewal_wo['CompletionDate'] = pd.to_datetime(all_renewal_wo['CompletionDate']).dt.date

all_renewal_wo.rename(columns={'FunctionLocation': 'floc_id'}, inplace=True)
all_renewal_wo.rename(columns={'CompletionDate': 'completion_date'}, inplace=True)

current_date = datetime.now().date()
all_renewal_wo = all_renewal_wo[(all_renewal_wo['completion_date'] <= current_date)]

all_renewal_wo = all_renewal_wo.sort_values(by='completion_date', ascending=False)
all_renewal_wo = all_renewal_wo.drop_duplicates(subset='floc_id', keep='first')

all_renewal_wo

C:\Users\R893859\AppData\Local\Temp\ipykernel_18768\3191190570.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_renewal_wo = pd.read_sql_query(sql_query, conn)


,floc_id,completion_date
1744,TO001156,2024-08-08
5611,CU100645,2024-08-08
8706,CU100047,2024-08-08
8709,BW-03ML,2024-08-08
1054,TO000570,2024-08-08
...,...,...
2660,N01-S009,2016-10-25
9609,LU000048,2016-10-25
2298,TO000338,2016-09-19
9997,TO000220,2016-08-10


In [3]:
sql_query = """
    SELECT DISTINCT
        FLOC_STRNO_FunctionalLocation,
        IFLOT_DATAB_StartupDate
    FROM
    myANALYTICS_SP.[workarea.silver.dimension.enterprise.asset].vw_Dim_FunctionalLocationStartupDate
    WHERE IFLOT_DATAB_StartupDate > 0
    AND FLOC_STRNO_FunctionalLocation IS NOT NULL
    AND FLOC_BUKRS_CompanyCode = '5000'
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';DATABASE=myANALYTICS_SP;Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
startupdate_df = pd.read_sql_query(sql_query, conn)

# df1.to_pickle("../pkl/work_order_all.pkl")
# Display the DataFrame
startupdate_df['IFLOT_DATAB_StartupDate'] = pd.to_datetime(startupdate_df['IFLOT_DATAB_StartupDate'])

startupdate_df.rename(columns={'FLOC_STRNO_FunctionalLocation': 'floc_id'}, inplace=True)
startupdate_df.rename(columns={'IFLOT_DATAB_StartupDate': 'startup_date'}, inplace=True)

startupdate_df

C:\Users\R893859\AppData\Local\Temp\ipykernel_18768\1586986036.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  startupdate_df = pd.read_sql_query(sql_query, conn)


,floc_id,startup_date
0,ST007470-05,2008-03-12
1,ST000980-91,2008-01-01
2,ST009690-01,2015-07-17
3,ST006360-30,2022-12-01
4,ST003915-13,2022-01-01
...,...,...
25990,GA-0908-00908G-SQ0040,2021-04-28
25991,ST007030-40,2006-01-01
25992,ST006030-33,2015-06-21
25993,ST005995-58,2023-11-01


In [4]:
startupdate_df

,floc_id,startup_date
0,ST007470-05,2008-03-12
1,ST000980-91,2008-01-01
2,ST009690-01,2015-07-17
3,ST006360-30,2022-12-01
4,ST003915-13,2022-01-01
...,...,...
25990,GA-0908-00908G-SQ0040,2021-04-28
25991,ST007030-40,2006-01-01
25992,ST006030-33,2015-06-21
25993,ST005995-58,2023-11-01


In [5]:
# merged_df1 = pd.concat([startupdate_df, all_renewal_wo], ignore_index=True)
merged_df1 = startupdate_df
merged_df1[merged_df1['floc_id'] == 'ST000770-49']

,floc_id,startup_date
10074,ST000770-49,2014-03-23


In [6]:
# all_structure_floc
sql_query = """
	-- Converting Civil Structures Asset Renewal from Python to SQL
	WITH DF1 AS(
		SELECT [FuncLocID]
			,[STRUCTURE_LENGTH]
			,[YEAR_OF_CONSTRUCTION]
			,[LinearRefPattern]
			,(CASE
					WHEN [ASSET_CLASS_TYPE] = 'BRIDGE' AND [SPAN_MATERIAL] = 'STEEL' THEN 'Steel Bridge'
					WHEN [ASSET_CLASS_TYPE] = 'BRIDGE' AND [SPAN_MATERIAL] = 'TIMBER' THEN 'Timber Bridge'
					ELSE NULL
				END) AS [Asset_Type_Bridge]
			,(CASE
					WHEN [STRUCTURE_TYPE] = 'REINFORCED CONCRETE BOXCULVERT' AND CAST(SUBSTRING(CAST([YEAR_OF_CONSTRUCTION] AS VARCHAR), 1, 4) AS FLOAT) < 2000 THEN 'Reinforced Concrete Box Culvert (RCBC) prior to year 2000'
					WHEN [STRUCTURE_TYPE] = 'REINFORCED CONCRETE BOXCULVERT' AND CAST(SUBSTRING(CAST([YEAR_OF_CONSTRUCTION] AS VARCHAR), 1, 4) AS FLOAT) >= 2000 THEN 'Reinforced Concrete Box Culvert (RCBC) designed to AS1597'
					ELSE NULL
				END) AS Asset_Type_Boxculvert
			,(CASE 
					WHEN [STRUCTURE_TYPE] = 'REINFORCED PIPE CULVERT' THEN 'Reinforced Concrete Pipe (RCP)'
					WHEN [STRUCTURE_TYPE] = 'CORRUGATED METAL PIPE' THEN 'Corrugated Metal Pipe (CMP)'
					WHEN [STRUCTURE_TYPE] != 'REINFORCED PIPE CULVERT' AND [SPAN_TYPE] = 'PIPE CULVERT' AND SPAN_MATERIAL LIKE '%CONCRETE%' THEN 'Concrete Pipe (Unreinforced)'
					ELSE NULL
				END) AS [Asset_Type_Pipe]
			,(CASE 
					WHEN [SPAN_MATERIAL] = 'CAST-IN-SITU CONCRETE' THEN 'Concrete In situ Arch'
					ELSE NULL
				END) AS [Asset_Type_Arch]
			,(CASE 
					WHEN [FuncLocID] LIKE 'RW%' THEN 'Concrete Retaining Wall'
					ELSE NULL
				END) AS [Asset_Type_Wall]
			,(CASE 
					WHEN SPAN_MATERIAL = 'STEEL' AND STRUCTURE_USAGE = 'PEDESTRIAN' AND TechnicalObjectTypeDesc = 'Bridges' THEN 'Pedestrian Overbridge (Steel)'
					ELSE NULL
				END) AS [Asset_Type_Overbridge]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.civil.structure.ringfenced].[vw_FLOCStructure]
		WHERE 1=1
		AND [UserStatusDesc]  = 'In Service'
	),

	DF2 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[STRUCTURE_LENGTH]
			,[YEAR_OF_CONSTRUCTION]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[STRUCTURE_LENGTH]
				,[YEAR_OF_CONSTRUCTION]
				,[LinearRefPattern]
				,CAST([Asset_Type_Bridge] AS VARCHAR(100)) AS [Asset_Type_Bridge]
				,CAST([Asset_Type_Boxculvert] AS VARCHAR(100)) AS [Asset_Type_Boxculvert]
				,CAST([Asset_Type_Pipe] AS VARCHAR(100)) AS [Asset_Type_Pipe]
				,CAST([Asset_Type_Arch] AS VARCHAR(100)) AS [Asset_Type_Arch]
				,CAST([Asset_Type_Wall] AS VARCHAR(100)) AS [Asset_Type_Wall]
				,CAST([Asset_Type_Overbridge] AS VARCHAR(100)) AS [Asset_Type_Overbridge]
			FROM DF1) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([Asset_Type_Bridge]
				,[Asset_Type_Boxculvert]
				,[Asset_Type_Pipe]
				,[Asset_Type_Arch]
				,[Asset_Type_Wall]
				,[Asset_Type_Overbridge])
			) AS unpvt
	)

	SELECT * FROM DF2
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_structure_floc = pd.read_sql_query(sql_query, conn)
all_structure_floc['YEAR_OF_CONSTRUCTION'] = pd.to_datetime(all_structure_floc['YEAR_OF_CONSTRUCTION'], format='%Y%m%d', errors='coerce')

all_structure_floc = all_structure_floc[['FuncLocID', 'ASSET', 'YEAR_OF_CONSTRUCTION']]
all_structure_floc['ASSET_GROUP'] = 'civil_structure'

all_structure_floc.rename(columns={'FuncLocID': 'floc_id'}, inplace=True)
all_structure_floc.rename(columns={'YEAR_OF_CONSTRUCTION': 'construction_year'}, inplace=True)

# # Display the DataFrame
# all_structure_floc

C:\Users\R893859\AppData\Local\Temp\ipykernel_18768\2254541309.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_structure_floc = pd.read_sql_query(sql_query, conn)


In [7]:
# all_civil_track_floc
sql_query = """
    -- Civil Track Asset Renewal Python Replication in SQL.
    WITH DF1 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE
                    WHEN LEFT_RAIL_HARDNESS LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Sml Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + LEFT_RAIL_HARDNESS + ' Curves' + ' (200 to 600 metre radius)'
                    WHEN LEFT_RAIL_HARDNESS NOT LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Sml Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + 'Curves' + ' (200 to 600 metre radius)'
                    WHEN LEFT_RAIL_HARDNESS LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Med Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + LEFT_RAIL_HARDNESS + ' Curves' + ' (601 to 1000 metre radius)'
                    WHEN LEFT_RAIL_HARDNESS NOT LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Med Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + 'Curves' + ' (601 to 1000 metre radius)'
                    WHEN LEFT_RAIL_HARDNESS LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Lrg Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + LEFT_RAIL_HARDNESS + ' Curves' + ' (1001 to 2500 metre radius)'
                    WHEN LEFT_RAIL_HARDNESS NOT LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Lrg Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + 'Curves' + ' (1001 to 2500 metre radius)'
                    WHEN Class = 'ZPM_TANGENTS' THEN LEFT_RAIL_MASS + ' Rail - ' + 'Straights'
                    ELSE NULL
                END) AS [ASSET_TYPE_RAIL]
                ,(CASE
                    WHEN SLEEPER_SIZE_VALUE = 'CON 28T' THEN '28.0 Tonne Axle Load PSC Sleepers'
                    WHEN SLEEPER_SIZE_VALUE = 'CON 22T' THEN '22.5 Tonne Axle Load PSC Sleepers'
                    WHEN SLEEPER_SIZE_VALUE = 'INTERSPERSED CONCRETE/TIMBER' OR SLEEPER_SIZE_VALUE = 'TIMBER' THEN 'Timber Sleepers'
                    WHEN SLEEPER_SIZE_VALUE = 'STEEL' THEN 'Steel Sleepers'
                    ELSE NULL
                END) AS [ASSET_TYPE_SLEEPERS]
                ,(CASE
                    WHEN SLEEPER_FASTENINGS LIKE '%PANDROL%' THEN 'Replace Pandrol E-Clip Fastenings'
                    WHEN SLEEPER_FASTENINGS = 'FIST-BTR' THEN 'Replace Fist Fastenings'
                    ELSE NULL
                END) AS [ASSET_TYPE_FASTENINGS]

        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.track.ringfenced].[vw_FLOCTrackFeature]
        WHERE UserStatusDesc = 'In Service'
    ),

    DF2 AS (
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_RAIL] AS VARCHAR(100)) AS [ASSET_TYPE_RAIL]
                ,CAST([ASSET_TYPE_SLEEPERS] AS VARCHAR(100)) AS [ASSET_TYPE_SLEEPERS]
                ,CAST([ASSET_TYPE_FASTENINGS] AS VARCHAR(100)) AS [ASSET_TYPE_FASTENINGS]
            FROM DF1) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_RAIL]
                ,[ASSET_TYPE_SLEEPERS]
                ,[ASSET_TYPE_FASTENINGS])
            ) AS unpvt
    ),

    DF3 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE
                    WHEN VEE_TYPE = 'SWINGNOSE' THEN ANGLE + ' (' + LEFT_RAIL_MASS + ') ' + 'Swing Nose Points and Crossing'
                    WHEN VEE_TYPE = 'RBM' THEN ANGLE + ' (' + LEFT_RAIL_MASS + ') ' + 'RBM Points and Crossing'
                    WHEN VEE_TYPE = 'FABRICATED' THEN ANGLE + ' (' + LEFT_RAIL_MASS + ') ' + 'Fabricated Points and Crossing'
                    WHEN VEE_TYPE = 'SPRINGWING' THEN ANGLE + ' (' + LEFT_RAIL_MASS + ') ' + 'Spring Wing Crossing'
                    WHEN VEE_TYPE IS NOT NULL THEN ANGLE + ' (' + LEFT_RAIL_MASS + ') ' + VEE_TYPE
                    ELSE NULL
                END) AS [ASSET_TYPE_POINTS_AND_CROSSING]
        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.track.rail.ringfenced].[vw_FLOCTurnoutExtended]
        WHERE UserStatusDesc = 'In Service'
    ),

    DF4 AS (
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_POINTS_AND_CROSSING] AS VARCHAR(100)) AS [ASSET_TYPE_POINTS_AND_CROSSING]
            FROM DF3) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_POINTS_AND_CROSSING])
            ) AS unpvt
    ),

    DF5 AS (

        SELECT [FuncLocID]
            ,'Insulated Rail Joints' AS [ASSET]
            ,'ASSET_TYPE_IRJS' AS [ASSET_TYPE]
            ,NULL AS FuncLocLinearLengthKM
            ,[LinearRefPattern]
        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.track.rail.ringfenced].[vw_FLOCIRJ]
        WHERE UserStatusDesc = 'In Service'
    ),

    RESULT AS (
        SELECT * FROM DF2
        UNION ALL
        SELECT * FROM DF4
        UNION ALL
        SELECT * FROM DF5
    )

    SELECT * FROM RESULT;
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_civil_track_floc = pd.read_sql_query(sql_query, conn)

all_civil_track_floc = all_civil_track_floc[['FuncLocID', 'ASSET']]
all_civil_track_floc['ASSET_GROUP'] = 'civil_track'

all_civil_track_floc.rename(columns={'FuncLocID': 'floc_id'}, inplace=True)

# # Display the DataFrame
# all_civil_track_floc

C:\Users\R893859\AppData\Local\Temp\ipykernel_18768\1161289048.py:120: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_civil_track_floc = pd.read_sql_query(sql_query, conn)


In [8]:
# all_civil_right_of_way_floc
sql_query = """
    -- Civil Right of Way Asset Renewal Python Conversion to SQL.
    WITH DF1 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE
                    WHEN TechnicalObjectTypeDesc = 'Public LX' THEN 'Public Level Xing Refurbishment (signage, drainage, road surface)'
                    WHEN TechnicalObjectTypeDesc = 'Maintenance LX' THEN 'Occupational & Maintenance Level Xing Refurbishment (signage, drainage, road surface)'
                    WHEN FuncLocName4 LIKE '%cattle%' THEN 'Cattle Xing Refurbishment (signage, drainage, road surfacing)'
                    ELSE NULL
                END) AS [ASSET_TYPE_CROSSING]
        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.operationalsystem.ringfenced].[vw_FLOCLevelCrossing]
        WHERE [UserStatusDesc]  = 'In Service'
    ),

    DF2 AS(
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_CROSSING] AS VARCHAR(100)) AS [ASSET_TYPE_CROSSING]
            FROM DF1) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_CROSSING])
            ) AS unpvt
    ),

    DF3 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE
                    WHEN AZJ_SG_EQUIPGRP LIKE '%pedestrian%' THEN 'Pedestrian Foot Xing Refurbishment (signage, drainage, road surface)'
                    ELSE NULL
                END) AS [ASSET_TYPE_CROSSING]
        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.electrical.ringfenced].[vw_FLOCElectrical]
        WHERE [UserStatusDesc]  = 'In Service'
    ),

    DF4 AS(
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_CROSSING] AS VARCHAR(100)) AS [ASSET_TYPE_CROSSING]
            FROM DF3) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_CROSSING])
            ) AS unpvt
    ),

    RESULT AS (
        SELECT * FROM DF2
        UNION ALL
        SELECT * FROM DF4
    )

    SELECT * FROM RESULT;
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_civil_right_of_way_floc = pd.read_sql_query(sql_query, conn)

all_civil_right_of_way_floc = all_civil_right_of_way_floc[['FuncLocID', 'ASSET']]
all_civil_right_of_way_floc['ASSET_GROUP'] = 'civil_rightofway'

all_civil_right_of_way_floc.rename(columns={'FuncLocID': 'floc_id'}, inplace=True)

# # Display the DataFrame
# all_civil_right_of_way_floc

C:\Users\R893859\AppData\Local\Temp\ipykernel_18768\4077560328.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_civil_right_of_way_floc = pd.read_sql_query(sql_query, conn)


In [9]:
# all_control_system_floc
sql_query = """
	-- Control Systems Asset Renewal Python Conversion to SQL.
	WITH DF1 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN [FuncLocName] LIKE '%signal gantry%' THEN 'Signal Gantry'
					ELSE NULL
				END) AS [ASSET_TYPE_GANTRY]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.civil.structure.ringfenced].[vw_FLOCStructure]
		WHERE [UserStatusDesc]  = 'In Service'
	),

	DF2 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_GANTRY] AS VARCHAR(100)) AS [ASSET_TYPE_GANTRY]
			FROM DF1) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_GANTRY])
			) AS unpvt
	),

	DF3 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN AZJ_SG_SYSGRP = 'TRAIN PROTECTION' AND AZJ_SG_EQUIPTYPE LIKE '%ATP%' THEN 'Train Protection Systems (ATP)'
					ELSE NULL
				END) AS [ASSET_TYPE_TRAIN_PROTECTION]
			,(CASE
					WHEN TechnicalObjectTypeDesc = 'Weather Station Syst' THEN 'Weather Station'
					ELSE NULL
				END) AS [ASSET_TYPE_WEATHER_STATION]
			,(CASE
					WHEN [AZJ_SG_SYSTYPE] = 'CROSSINGS' AND [AZJ_SG_EQUIPTYPE] LIKE 'RMS%' THEN 'Level Crossing Monitors'
					ELSE NULL
				END) AS [ASSET_TYPE_LX_MONITORS]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.track.ringfenced].[vw_FLOCTrainDetection]
		WHERE [UserStatusDesc]  = 'In Service'
	),
	
	DF4 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_TRAIN_PROTECTION] AS VARCHAR(100)) AS [ASSET_TYPE_TRAIN_PROTECTION]
				,CAST([ASSET_TYPE_WEATHER_STATION] AS VARCHAR(100)) AS [ASSET_TYPE_WEATHER_STATION]
				,CAST([ASSET_TYPE_LX_MONITORS] AS VARCHAR(100)) AS [ASSET_TYPE_LX_MONITORS]
			FROM DF3) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_TRAIN_PROTECTION],
				[ASSET_TYPE_WEATHER_STATION],
				[ASSET_TYPE_LX_MONITORS])
			) AS unpvt
	),

	DF5 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN QR_CONFIG = 'LED SIGNAL' THEN 'LED Signal Modules'
					ELSE NULL
				END) AS [ASSET_TYPE_SIGNAL_MODULES]
			,(CASE
					WHEN FuncLocName LIKE '%s2%' AND FuncLocName LIKE '%telemetry%' THEN 'S2 Telemetry Unit'
					ELSE NULL
				END) AS [ASSET_TYPE_TELEMETRY_UNIT]
			,(CASE
					WHEN AZJ_SG_SYSTYPE = 'WHEEL IMPACT LOAD DETECTOR' THEN 'Wheel Impact Load Detector (WILD)'
					WHEN TechnicalObjectTypeDesc = 'Hot Bearing / Wheel' AND AZJ_SG_EQUIPTYPE LIKE '%detector%' THEN 'Hot Bearing and Hot Wheel Detectors'
					ELSE NULL
				END) AS [ASSET_TYPE_DETECTOR]
			,(CASE
					WHEN [AZJ_SG_EQUIPTYPE] = 'POLE' THEN 'Radio Poles'
					ELSE NULL
				END) AS [ASSET_TYPE_POLE]
			,(CASE
					WHEN AZJ_SG_EQUIPGRP = 'ELECTRIC POINTS' OR AZJ_SG_EQUIPGRP = 'HYDRAULIC POINTS' OR AZJ_SG_EQUIPGRP = 'MECHANICAL POINTS' THEN 'Points Machines'
					ELSE NULL
				END) AS [ASSET_TYPE_POINTS]
			,(CASE
					WHEN TechnicalObjectTypeDesc = 'Alternator Control E' AND WorkCenterDesc LIKE '%Control Systems%' AND AZJ_SG_EQUIPGRP = 'STANDBY SUPPLY' THEN 'Diesel Standby Alternators'
					ELSE NULL
				END) AS [ASSET_TYPE_ALTERNATORS]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.ringfenced].[vw_FLOCTelemetry]
		WHERE [UserStatusDesc]  = 'In Service'
	),

	DF6 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_SIGNAL_MODULES] AS VARCHAR(100)) AS [ASSET_TYPE_SIGNAL_MODULES]
				,CAST([ASSET_TYPE_TELEMETRY_UNIT] AS VARCHAR(100)) AS [ASSET_TYPE_TELEMETRY_UNIT]
				,CAST([ASSET_TYPE_DETECTOR] AS VARCHAR(100)) AS [ASSET_TYPE_DETECTOR]
				,CAST([ASSET_TYPE_POLE] AS VARCHAR(100)) AS [ASSET_TYPE_POLE]
				,CAST([ASSET_TYPE_POLE] AS VARCHAR(100)) AS [ASSET_TYPE_POINTS]
				,CAST([ASSET_TYPE_POLE] AS VARCHAR(100)) AS [ASSET_TYPE_ALTERNATORS]
			FROM DF5) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_SIGNAL_MODULES],
				[ASSET_TYPE_TELEMETRY_UNIT],
				[ASSET_TYPE_DETECTOR],
				[ASSET_TYPE_POLE],
				[ASSET_TYPE_POINTS],
				[ASSET_TYPE_ALTERNATORS])
			) AS unpvt
	),

	DF7 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'UPS' THEN 'UPS'
					ELSE NULL
				END) AS [ASSET_TYPE_UPS]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'RELAY INTERLOCKING' THEN 'Relay Interlockings'
					ELSE NULL
				END) AS [ASSET_TYPE_INTERLOCKINGS]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'DC TRACK CIRCUIT' THEN 'DC and Impulse Track Circuits'
					WHEN AZJ_SG_EQUIPTYPE = 'JS TRACK CIRCUIT' THEN 'Jointless Track Circuits'
					ELSE NULL
				END) AS [ASSET_TYPE_TRACK_CIRCUITS]
			,(CASE
					WHEN TechnicalObjectTypeDesc = 'Axle Counter' THEN 'Axle Counters'
					ELSE NULL
				END) AS [ASSET_TYPE_AXLE_COUNTERS]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'DRAGGING EQUIPMENT DETECTOR' THEN 'Dragging Equipment Detector'
					ELSE NULL
				END) AS [ASSET_TYPE_DRAGGING_EQUIPMENT]
			,(CASE
					WHEN AZJ_SG_EQUIPGRP = 'ELECTRONIC WEIGHBRIDGE' THEN 'Weighbridges'
					ELSE NULL
				END) AS [ASSET_TYPE_WEIGHBRIDGES]
			,(CASE
					WHEN TechnicalObjectTypeDesc = 'Level Crossing' AND AZJ_SG_EQUIPTYPE LIKE '%camera%' THEN 'Level Crossings Cameras'
					ELSE NULL
				END) AS [ASSET_TYPE_LX_CAMERAS]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'LX PROTECTION SYSTEM' THEN 'Over Height Level Crossing Protection'
					ELSE NULL
				END) AS [ASSET_TYPE_LX_PROTECTION]
			,(CASE
					WHEN AZJ_SG_SYSGRP = 'CQCN DATA NETWORK' AND (AZJ_SG_EQUIPGRP LIKE '%switch%' OR AZJ_SG_EQUIPGRP LIKE '%router%') THEN 'Data - LAN (Switch & Routers)'
					ELSE NULL
				END) AS [ASSET_TYPE_DATA_LAN]
			,(CASE
					WHEN AZJ_SG_SYSGRP = 'RADIO SYSTEMS' AND AZJ_SG_SYSTYPE = 'TETRA' THEN 'Radio Systems - TETRA'
					ELSE NULL
				END) AS [ASSET_TYPE_RADIO_SYSTEMS]
			,(CASE
					WHEN AZJ_SG_SYSGRP = 'RADIO SYSTEMS' AND AZJ_SG_SYSID LIKE '%ded%' THEN 'Radio DEDs'
					ELSE NULL
				END) AS [ASSET_TYPE_RADIO_DEDS]
			,(CASE
					WHEN AZJ_SG_EQUIPGRP LIKE '%antenna%' AND AZJ_SG_EQUIPTYPE LIKE '%pole%' THEN 'Radio Poles'
					ELSE NULL
				END) AS [ASSET_TYPE_RADIO_POLES]
			,(CASE
					WHEN TechnicalObjectTypeDesc = 'DMR' THEN 'Digital Microwave Radio (DMR)'
					ELSE NULL
				END) AS [ASSET_TYPE_DMR]
			,(CASE
					WHEN AZJ_SG_EQUIPGRP = 'SDH' OR AZJ_SG_EQUIPGRP = 'PDH' THEN 'Active Terminal Equipment (PDH, SDH)'
					ELSE NULL
				END) AS [ASSET_TYPE_TERMINAL_EQUIPMENT]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'COMMS EQUIPMENT ROOM' THEN 'Communication Equipment Rooms'
					ELSE NULL
				END) AS [ASSET_TYPE_EQUIPMENT_ROOMS]
			,(CASE
					WHEN AZJ_SG_EQUIPGRP = 'HOUSING' AND WorkCenterDesc LIKE '%control systems%' THEN 'Communication Housings'
					ELSE NULL
				END) AS [ASSET_TYPE_COMMUNICATION_HOUSINGS]
			,(CASE
					WHEN FuncLocName LIKE '%50v battery%' THEN '50V DC Power Supplies'
					ELSE NULL
				END) AS [ASSET_TYPE_DC_POWER_SUPPLIES]
			,(CASE
					WHEN FuncLocName LIKE '%battery bank%' THEN 'DC Battery Banks'
					ELSE NULL
				END) AS [ASSET_TYPE_BATTERY_BANKS]
			,(CASE
					WHEN AZJ_SG_SYSTYPE = 'UTC' OR AZJ_SG_SYSTYPE = 'DTC' THEN 'UTC & DTC'
					ELSE NULL
				END) AS [ASSET_TYPE_UTC_AND_DTC]
			,(CASE
					WHEN [AZJ_SG_EQUIPGRP] = 'CABLES' AND [AZJ_SG_EQUIPTYPE] = 'SINGLEMODE O/FIBRE' OR [AZJ_SG_EQUIPTYPE] = 'SINGLEMODE OPGW CABLE' THEN 'New Fibre Optic Cable' --Should be 'Original Fibre Optic Cable (installed 1985)' based on install date, that isn't here... The old ones have 30-year life, so should be replaced by now if kept to schedule.
					ELSE NULL
				END) AS [ASSET_TYPE_CABLES]
			,(CASE
					WHEN [AZJ_SG_EQUIPTYPE] = 'GUYED STEEL LATTICE TOWER' OR [AZJ_SG_EQUIPTYPE] = 'STEEL LATTICE TOWER' THEN 'Tower (Radio or Microwave)'
					ELSE NULL
				END) AS [ASSET_TYPE_TOWER]
			,(CASE
					WHEN FuncLocID LIKE 'LX%' AND FuncLocID = FuncLocID6 THEN 'Level Crossings (Software)'
					ELSE NULL
				END) AS [ASSET_TYPE_LX_SOFTWARE]
			,(CASE
					WHEN AZJ_SG_SYSTYPE = 'HOUSINGS' AND ASSET_OWNER = 'AURIZON - NETWORK' AND WorkCenterDesc LIKE '%Control%' AND AZJ_SG_SYSID LIKE '%HOUSING%' THEN 'Communication Housings'
					ELSE NULL
				END) AS [ASSET_TYPE_COMMUNICATION_HOUSING]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.electrical.ringfenced].[vw_FLOCElectrical]
		WHERE [UserStatusDesc]  = 'In Service'
	),

	DF8 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_UPS] AS VARCHAR(100)) AS [ASSET_TYPE_UPS]
				,CAST([ASSET_TYPE_INTERLOCKINGS] AS VARCHAR(100)) AS [ASSET_TYPE_INTERLOCKINGS]
				,CAST([ASSET_TYPE_TRACK_CIRCUITS] AS VARCHAR(100)) AS [ASSET_TYPE_TRACK_CIRCUITS]
				,CAST([ASSET_TYPE_AXLE_COUNTERS] AS VARCHAR(100)) AS [ASSET_TYPE_AXLE_COUNTERS]
				,CAST([ASSET_TYPE_DRAGGING_EQUIPMENT] AS VARCHAR(100)) AS [ASSET_TYPE_DRAGGING_EQUIPMENT]
				,CAST([ASSET_TYPE_WEIGHBRIDGES] AS VARCHAR(100)) AS [ASSET_TYPE_WEIGHBRIDGES]
				,CAST([ASSET_TYPE_LX_CAMERAS] AS VARCHAR(100)) AS [ASSET_TYPE_LX_CAMERAS]
				,CAST([ASSET_TYPE_LX_PROTECTION] AS VARCHAR(100)) AS [ASSET_TYPE_LX_PROTECTION]
				,CAST([ASSET_TYPE_DATA_LAN] AS VARCHAR(100)) AS [ASSET_TYPE_DATA_LAN]
				,CAST([ASSET_TYPE_RADIO_SYSTEMS] AS VARCHAR(100)) AS [ASSET_TYPE_RADIO_SYSTEMS]
				,CAST([ASSET_TYPE_RADIO_DEDS] AS VARCHAR(100)) AS [ASSET_TYPE_RADIO_DEDS]
				,CAST([ASSET_TYPE_RADIO_POLES] AS VARCHAR(100)) AS [ASSET_TYPE_RADIO_POLES]
				,CAST([ASSET_TYPE_DMR] AS VARCHAR(100)) AS [ASSET_TYPE_DMR]
				,CAST([ASSET_TYPE_TERMINAL_EQUIPMENT] AS VARCHAR(100)) AS [ASSET_TYPE_TERMINAL_EQUIPMENT]
				,CAST([ASSET_TYPE_EQUIPMENT_ROOMS] AS VARCHAR(100)) AS [ASSET_TYPE_EQUIPMENT_ROOMS]
				,CAST([ASSET_TYPE_COMMUNICATION_HOUSINGS] AS VARCHAR(100)) AS [ASSET_TYPE_COMMUNICATION_HOUSINGS]
				,CAST([ASSET_TYPE_DC_POWER_SUPPLIES] AS VARCHAR(100)) AS [ASSET_TYPE_DC_POWER_SUPPLIES]
				,CAST([ASSET_TYPE_BATTERY_BANKS] AS VARCHAR(100)) AS [ASSET_TYPE_BATTERY_BANKS]
				,CAST([ASSET_TYPE_UTC_AND_DTC] AS VARCHAR(100)) AS [ASSET_TYPE_UTC_AND_DTC]
				,CAST([ASSET_TYPE_CABLES] AS VARCHAR(100)) AS [ASSET_TYPE_CABLES]
				,CAST([ASSET_TYPE_TOWER] AS VARCHAR(100)) AS [ASSET_TYPE_TOWER]
				,CAST([ASSET_TYPE_LX_SOFTWARE] AS VARCHAR(100)) AS [ASSET_TYPE_LX_SOFTWARE]
				,CAST([ASSET_TYPE_COMMUNICATION_HOUSING] AS VARCHAR(100)) AS [ASSET_TYPE_COMMUNICATION_HOUSING]
			FROM DF7) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_UPS],
				[ASSET_TYPE_INTERLOCKINGS],
				[ASSET_TYPE_TRACK_CIRCUITS],
				[ASSET_TYPE_AXLE_COUNTERS],
				[ASSET_TYPE_DRAGGING_EQUIPMENT],
				[ASSET_TYPE_WEIGHBRIDGES],
				[ASSET_TYPE_LX_CAMERAS],
				[ASSET_TYPE_LX_PROTECTION],
				[ASSET_TYPE_DATA_LAN],
				[ASSET_TYPE_RADIO_SYSTEMS],
				[ASSET_TYPE_RADIO_DEDS],
				[ASSET_TYPE_RADIO_POLES],
				[ASSET_TYPE_DMR],
				[ASSET_TYPE_TERMINAL_EQUIPMENT],
				[ASSET_TYPE_EQUIPMENT_ROOMS],
				[ASSET_TYPE_COMMUNICATION_HOUSINGS],
				[ASSET_TYPE_DC_POWER_SUPPLIES],
				[ASSET_TYPE_BATTERY_BANKS],
				[ASSET_TYPE_UTC_AND_DTC],
				[ASSET_TYPE_CABLES],
				[ASSET_TYPE_TOWER],
				[ASSET_TYPE_LX_SOFTWARE],
				[ASSET_TYPE_COMMUNICATION_HOUSING])
			) AS unpvt
	),

	DF9 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN FuncLocID LIKE 'LX%' AND FuncLocID = FuncLocID6 THEN 'Level Crossing (Hardware)'
					ELSE NULL
				END) AS [ASSET_TYPE_LX_HARDWARE]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.operationalsystem.ringfenced].[vw_FLOCLevelCrossingExtended]
		WHERE [UserStatusDesc]  = 'In Service'
	),

	DF10 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_LX_HARDWARE] AS VARCHAR(100)) AS [ASSET_TYPE_LX_HARDWARE]
			FROM DF9) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_LX_HARDWARE])
			) AS unpvt
	),

	DF11 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN AZJ_SG_SYSTYPE = 'INTERLOCKING' AND WorkCenterDesc LIKE '%Control%' THEN 'Processor Based Interlockings'
					ELSE NULL
				END) AS [ASSET_TYPE_INTERLOCKING]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.signalling.ringfenced].[vw_FLOCSignal] 
		WHERE [UserStatusDesc]  = 'In Service'
	),

	DF12 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_INTERLOCKING] AS VARCHAR(100)) AS [ASSET_TYPE_INTERLOCKING]
			FROM DF11) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_INTERLOCKING])
			) AS unpvt
	),

	RESULT AS (
		SELECT * FROM DF2
		UNION ALL
		SELECT * FROM DF4
		UNION ALL
		SELECT * FROM DF6
		UNION ALL
		SELECT * FROM DF8
		UNION ALL
		SELECT * FROM DF10
		UNION ALL
		SELECT * FROM DF12
	)

	SELECT * FROM RESULT;
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_control_system_floc = pd.read_sql_query(sql_query, conn)

all_control_system_floc = all_control_system_floc[['FuncLocID', 'ASSET']]
all_control_system_floc['ASSET_GROUP'] = 'control_systems'

all_control_system_floc.rename(columns={'FuncLocID': 'floc_id'}, inplace=True)

# # # Display the DataFrame
# all_control_system_floc

C:\Users\R893859\AppData\Local\Temp\ipykernel_18768\502845613.py:384: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_control_system_floc = pd.read_sql_query(sql_query, conn)


In [10]:
# all_traction_power_equipment_floc
sql_query = """
    -- Traction Power Equipment Asset Renewal Python Replication in SQL.
    WITH DF1 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE 
                    WHEN AZJ_SG_EQUIPTYPE = 'HARMONIC FILTER' THEN 'Harmonic Filters'
                    ELSE NULL
                END) AS [ASSET_TYPE_FILTER]
            ,(CASE 
                    WHEN TechnicalObjectTypeDesc = 'Transformer' AND AZJ_SG_SYSGRP = 'TRACTION POWER SYSTEM' THEN 'Traction Power Supply Transformers'
                    ELSE NULL
                END) AS [ASSET_TYPE_TRANSFORMER]
            ,(CASE 
                    WHEN AZJ_SG_EQUIPTYPE = 'AUTOTRANSFORMER' THEN 'Autotransformers'
                    ELSE NULL
                END) AS [ASSET_TYPE_AUTOTRANSFORMER]
            ,(CASE 
                    WHEN AZJ_SG_EQUIPTYPE = 'AUXILARY SUPPLY TRANSFORMER' THEN 'Auxiliary Supply Transformers'
                    ELSE NULL
                END) AS [ASSET_TYPE_AUXILIARY_SUPPLY_TRANSFORMER]
            ,(CASE 
                    WHEN TechnicalObjectTypeDesc = 'HV Switchgear' AND AZJ_SG_SYSGRP = 'TRACTION POWER SYSTEM' THEN 'Traction Power Systems Switchgear'
                    ELSE NULL
                END) AS [ASSET_TYPE_SWITCHGEAR]
            ,(CASE 
                    WHEN TechnicalObjectTypeDesc = 'Isolators' THEN 'Isolator'
                    ELSE NULL
                END) AS [ASSET_TYPE_ISOLATOR]
            ,(CASE 
                    WHEN TechnicalObjectTypeDesc = 'Neutral Section' THEN 'Neutral Sections'
                    WHEN TechnicalObjectTypeDesc = 'Section Insulator' THEN 'Section Insulators'
                    ELSE NULL
                END) AS [ASSET_TYPE_SECTION]
            ,(CASE 
                    WHEN AZJ_SG_SYSGRP = 'TRACTION OVERHEAD SYSTEM' THEN 'Overhead Line System'
                    ELSE NULL
                END) AS [ASSET_TYPE_OVERHEAD]
            ,(CASE 
                    WHEN AZJ_SG_SYSGRP = 'SCADA' AND AZJ_SG_SYSTYPE = 'POWER SUPERVISORY' THEN 'Traction SCADA'
                    ELSE NULL
                END) AS [ASSET_TYPE_SCADA]
        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.electrical.ringfenced].[vw_FLOCElectrical]
        WHERE [UserStatusDesc]  = 'In Service'
    ),

    DF2 AS(
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_FILTER] AS VARCHAR(100)) AS [ASSET_TYPE_FILTER]
                ,CAST([ASSET_TYPE_TRANSFORMER] AS VARCHAR(100)) AS [ASSET_TYPE_TRANSFORMER]
                ,CAST([ASSET_TYPE_AUTOTRANSFORMER] AS VARCHAR(100)) AS [ASSET_TYPE_AUTOTRANSFORMER]
                ,CAST([ASSET_TYPE_AUXILIARY_SUPPLY_TRANSFORMER] AS VARCHAR(100)) AS [ASSET_TYPE_AUXILIARY_SUPPLY_TRANSFORMER]
                ,CAST([ASSET_TYPE_SWITCHGEAR] AS VARCHAR(100)) AS [ASSET_TYPE_SWITCHGEAR]
                ,CAST([ASSET_TYPE_ISOLATOR] AS VARCHAR(100)) AS [ASSET_TYPE_ISOLATOR]
                ,CAST([ASSET_TYPE_SECTION] AS VARCHAR(100)) AS [ASSET_TYPE_SECTION]
                ,CAST([ASSET_TYPE_OVERHEAD] AS VARCHAR(100)) AS [ASSET_TYPE_OVERHEAD]
                ,CAST([ASSET_TYPE_SCADA] AS VARCHAR(100)) AS [ASSET_TYPE_SCADA]
            FROM DF1) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_FILTER]
                ,[ASSET_TYPE_TRANSFORMER]
                ,[ASSET_TYPE_AUTOTRANSFORMER]
                ,[ASSET_TYPE_AUXILIARY_SUPPLY_TRANSFORMER]
                ,[ASSET_TYPE_SWITCHGEAR]
                ,[ASSET_TYPE_ISOLATOR]
                ,[ASSET_TYPE_SECTION]
                ,[ASSET_TYPE_OVERHEAD]
                ,[ASSET_TYPE_SCADA])
            ) AS unpvt
    ),

    DF3 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE 
                    WHEN Characteristic = 'ZPM_ISOLATOR_TYPE' AND ([Value] = 'MANUAL' OR [Value] = 'MOTORISED') THEN 'Motorised Isolator Motor Units'
                    ELSE NULL
                END) AS [ASSET_TYPE_MOTORISED_ISOLATORS]

        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.ringfenced].[vw_FLOCEQUICharacteristics]
        WHERE [UserStatusDesc]  = 'In Service'
    ),

    DF4 AS(
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_MOTORISED_ISOLATORS] AS VARCHAR(100)) AS [ASSET_TYPE_MOTORISED_ISOLATORS]
            FROM DF3) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_MOTORISED_ISOLATORS])
            ) AS unpvt
    ),

    RESULT AS (
        SELECT * FROM DF2
        UNION ALL
        SELECT * FROM DF4
    )

    SELECT * FROM RESULT
"""
# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_traction_power_equipment_floc = pd.read_sql_query(sql_query, conn)

all_traction_power_equipment_floc = all_traction_power_equipment_floc[['FuncLocID', 'ASSET']]
all_traction_power_equipment_floc['ASSET_GROUP'] = 'traction_power_equipment'

all_traction_power_equipment_floc.rename(columns={'FuncLocID': 'floc_id'}, inplace=True)

# # # Display the DataFrame
# all_traction_power_equipment_floc

C:\Users\R893859\AppData\Local\Temp\ipykernel_18768\816844843.py:129: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_traction_power_equipment_floc = pd.read_sql_query(sql_query, conn)


In [11]:
# # Getting all FAR assets 'Asset Capitalized on' dates
# file_path = r'C:\Users\R893859\Aurizon Operations Limited\Data, Reporting & Analytics - DS08 - NSAP2\Data\Fixed Asset Register\09 - March 2024 FAR.xlsx'

# df1 = pd.read_excel(file_path)

# filtered_df = df1[df1['Company code'] == 5000]
# filtered_df = filtered_df[filtered_df['Functional loc.'].str.len() > 2]

# filtered_df['Asset Number'] = filtered_df['Asset Number'].astype(int)

# filtered_df['Order. dep. Start'] = pd.to_datetime(filtered_df['Order. dep. Start'], format='%d.%m.%Y')
# filtered_df['Asset Capitalized on'] = pd.to_datetime(filtered_df['Asset Capitalized on'], format='%d.%m.%Y')
# filtered_df['Asset First acq. on'] = pd.to_datetime(filtered_df['Asset First acq. on'], format='%d.%m.%Y')
# filtered_df['Asset Deactivation date'] = pd.to_datetime(filtered_df['Asset Deactivation date'], format='%d.%m.%Y', errors='coerce')

# filtered_df = filtered_df[
#     [
#         'Asset Number', 
#         'Sub number', 
#         'Asset Sub name', 
#         'Functional loc.', 
#         'LTD Cost', 
#         # 'Order. dep. Start',
#         'Asset Capitalized on',
#         'Asset Class text'
#         # 'Asset First acq. on'
#     ]
# ]
# # filtered_df = filtered_df[filtered_df['Company code'] == 'UNKNOWN']

# grouped_df = filtered_df.groupby(['Asset Number', 'Functional loc.', 'Asset Capitalized on', 'Asset Class text'], as_index=False)['LTD Cost'].sum()
# far_df = grouped_df[['Functional loc.', 'Asset Capitalized on', 'Asset Class text']]
# far_df.rename(columns={'Functional loc.': 'floc_id'}, inplace=True)

# far_df

In [12]:
# num_duplicates = far_df.duplicated('floc_id', keep=False).sum()

# print(f"Number of duplicate rows: {num_duplicates}")

# # Showing duplicates only:
# far_df[far_df.duplicated('floc_id', keep=False)].sort_values(by='floc_id')

In [13]:
# far_df['Asset Capitalized on'] = pd.to_datetime(far_df['Asset Capitalized on'])

# idx = far_df.groupby(['floc_id', 'Asset Class text'])['Asset Capitalized on'].idxmin()

# filtered_df = far_df.loc[idx]

# filtered_df = filtered_df.reset_index(drop=True)

# filtered_df

In [14]:
# num_duplicates = filtered_df.duplicated('floc_id', keep=False).sum()

# print(f"Number of duplicate rows: {num_duplicates}")

# # Showing duplicates only:
# filtered_df[filtered_df.duplicated('floc_id', keep=False)].sort_values(by='floc_id')

In [15]:
all_renewal_floc = pd.concat([all_structure_floc, all_civil_track_floc, all_civil_right_of_way_floc, all_control_system_floc, all_traction_power_equipment_floc], ignore_index=True)
all_renewal_floc

,floc_id,ASSET,construction_year,ASSET_GROUP
0,BR101296,Steel Bridge,1980-01-01,civil_structure
1,BR101296,Pedestrian Overbridge (Steel),1980-01-01,civil_structure
2,BR100452,Steel Bridge,1973-01-01,civil_structure
3,BR101653,Concrete In situ Arch,2014-11-15,civil_structure
4,BR101734,Reinforced Concrete Box Culvert (RCBC) prior t...,1983-01-01,civil_structure
...,...,...,...,...
54170,ST000040-05,Motorised Isolator Motor Units,NaT,traction_power_equipment
54171,ST000040-07,Motorised Isolator Motor Units,NaT,traction_power_equipment
54172,ST000160-08,Motorised Isolator Motor Units,NaT,traction_power_equipment
54173,ST002140-08,Motorised Isolator Motor Units,NaT,traction_power_equipment


In [36]:
# merged_df2 = pd.merge(all_renewal_floc, startupdate_df, how='left', left_on='floc_id', right_on='floc_id')
# merged_df2 = pd.concat([all_renewal_floc, startupdate_df], ignore_index=True)
merged_df2 = pd.merge(all_renewal_floc, startupdate_df, how='outer', on='floc_id')

merged_df2

,floc_id,ASSET,construction_year,ASSET_GROUP,startup_date
0,AR-01CR-CHCH,NaN,NaT,NaN,2021-01-01
1,AR-01CR-CHCH-ST0010,NaN,NaT,NaN,2021-01-01
2,AR-01ML,NaN,NaT,NaN,2021-01-01
3,AR-01ML-JBJB,NaN,NaT,NaN,2021-01-01
4,AR-01ML-JBJB-TO0010,NaN,NaT,NaN,2021-01-01
...,...,...,...,...,...
70592,TO004063,NaN,NaT,NaN,2021-01-01
70593,TO004064,NaN,NaT,NaN,2021-01-01
70594,TO004065,NaN,NaT,NaN,2021-01-01
70595,TO004067-01,NaN,NaT,NaN,2023-11-01


In [17]:
# merged_df2[merged_df2['floc_id'] == 'ST000770-49']

In [18]:
# Getting all FLOC from [vw_Dim_FunctionalLocation] where CompanyCode = 5000
sql_query = """
    SELECT
        FLOC_STRNO_FunctionalLocation,
        FLOC_PLTXT_FunctionalLocationDescr,
        FLOC_EARTX_TechnicalObjectTypeDescr,
        FLOC_STORT_LocationDescr
    FROM [myANALYTICS_SP].[silver.dimension.enterprise.asset].[vw_Dim_FunctionalLocation]
	WHERE FLOC_BUKRS_CompanyCode = '5000'
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_floc_no_age = pd.read_sql_query(sql_query, conn)

# Display the DataFrame
all_floc_no_age

C:\Users\R893859\AppData\Local\Temp\ipykernel_18768\3742737230.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_floc_no_age = pd.read_sql_query(sql_query, conn)


,FLOC_STRNO_FunctionalLocation,FLOC_PLTXT_FunctionalLocationDescr,FLOC_EARTX_TechnicalObjectTypeDescr,FLOC_STORT_LocationDescr
0,N01-S002-C217-L0963,WulkarakaX - Yarraman,None,None
1,N01-S002-C232,Corridor: Petrie to Kippa Ring,None,None
2,N01-S010-C078-L0147,Auckland Point Angle,Corridor Objects,None
3,N01-S001-C127-L0331,Comalco B/L JCT - Fishermans Landing B/L,Corridor Objects,None
4,N01-S001-C233,Corridor: Comalco Balloon Loop,Corridor Objects,None
...,...,...,...,...
174808,N01-S014-C210,Corridor: Westgate to Quilpie,None,None
174809,N01-S014-C210-L0121,Westgate Fork,None,None
174810,N01-S014-C210-L0716,WestgateX - Cooladdi,None,None
174811,N01-S014-C210-L0717,CooladdiX - Quilpie,None,None


In [37]:
all_floc_with_age = pd.merge(all_floc_no_age, merged_df2, how='left', left_on='FLOC_STRNO_FunctionalLocation', right_on='floc_id')

# all_floc_with_age = all_floc_with_age[['FLOC_STRNO_FunctionalLocation', 'FLOC_PLTXT_FunctionalLocationDescr', 'FLOC_EARTX_TechnicalObjectTypeDescr', 'FLOC_STORT_LocationDescr', 'ASSET', 'ASSET_GROUP', 'construction_year', 'startup_date', 'completion_date']]
all_floc_with_age = all_floc_with_age[['FLOC_STRNO_FunctionalLocation', 'FLOC_PLTXT_FunctionalLocationDescr', 'FLOC_EARTX_TechnicalObjectTypeDescr', 'FLOC_STORT_LocationDescr', 'ASSET', 'ASSET_GROUP', 'construction_year', 'startup_date']]

all_floc_with_age.rename(columns={'ASSET': 'asset_type'}, inplace=True)
all_floc_with_age.rename(columns={'ASSET_GROUP': 'asset_group'}, inplace=True)

all_floc_with_age
# all_floc_with_age[all_floc_with_age['FLOC_STRNO_FunctionalLocation'] == 'ST000770-49']

,FLOC_STRNO_FunctionalLocation,FLOC_PLTXT_FunctionalLocationDescr,FLOC_EARTX_TechnicalObjectTypeDescr,FLOC_STORT_LocationDescr,asset_type,asset_group,construction_year,startup_date
0,N01-S002-C217-L0963,WulkarakaX - Yarraman,None,None,NaN,NaN,NaT,NaT
1,N01-S002-C232,Corridor: Petrie to Kippa Ring,None,None,NaN,NaN,NaT,NaT
2,N01-S010-C078-L0147,Auckland Point Angle,Corridor Objects,None,NaN,NaN,NaT,NaT
3,N01-S001-C127-L0331,Comalco B/L JCT - Fishermans Landing B/L,Corridor Objects,None,NaN,NaN,NaT,NaT
4,N01-S001-C233,Corridor: Comalco Balloon Loop,Corridor Objects,None,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...
199648,N01-S014-C210,Corridor: Westgate to Quilpie,None,None,NaN,NaN,NaT,NaT
199649,N01-S014-C210-L0121,Westgate Fork,None,None,NaN,NaN,NaT,NaT
199650,N01-S014-C210-L0716,WestgateX - Cooladdi,None,None,NaN,NaN,NaT,NaT
199651,N01-S014-C210-L0717,CooladdiX - Quilpie,None,None,NaN,NaN,NaT,NaT


In [38]:
all_floc_with_age = all_floc_with_age.drop_duplicates()
all_floc_with_age

,FLOC_STRNO_FunctionalLocation,FLOC_PLTXT_FunctionalLocationDescr,FLOC_EARTX_TechnicalObjectTypeDescr,FLOC_STORT_LocationDescr,asset_type,asset_group,construction_year,startup_date
0,N01-S002-C217-L0963,WulkarakaX - Yarraman,None,None,NaN,NaN,NaT,NaT
1,N01-S002-C232,Corridor: Petrie to Kippa Ring,None,None,NaN,NaN,NaT,NaT
2,N01-S010-C078-L0147,Auckland Point Angle,Corridor Objects,None,NaN,NaN,NaT,NaT
3,N01-S001-C127-L0331,Comalco B/L JCT - Fishermans Landing B/L,Corridor Objects,None,NaN,NaN,NaT,NaT
4,N01-S001-C233,Corridor: Comalco Balloon Loop,Corridor Objects,None,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...
199648,N01-S014-C210,Corridor: Westgate to Quilpie,None,None,NaN,NaN,NaT,NaT
199649,N01-S014-C210-L0121,Westgate Fork,None,None,NaN,NaN,NaT,NaT
199650,N01-S014-C210-L0716,WestgateX - Cooladdi,None,None,NaN,NaN,NaT,NaT
199651,N01-S014-C210-L0717,CooladdiX - Quilpie,None,None,NaN,NaN,NaT,NaT


In [40]:
# Check for rows where all three date columns are missing
# mask = all_floc_with_age[['construction_year', 'startup_date', 'completion_date']].isnull().all(axis=1)
mask = all_floc_with_age[['construction_year', 'startup_date']].isnull().all(axis=1)

# Group by 'asset_group' and count items with missing dates
group_counts = all_floc_with_age.groupby('asset_group').size().reset_index(name='unique_values')
missing_date_counts = all_floc_with_age[mask].groupby('asset_group').size().reset_index(name='no_dates')

# Merge the two results
result = pd.merge(group_counts, missing_date_counts, on='asset_group', how='outer').fillna(0)

# Ensure the column data types are correct
result['unique_values'] = result['unique_values'].astype(int)
result['no_dates'] = result['no_dates'].astype(int)
result['yes_dates'] = result['unique_values'] - result['no_dates']

result

,asset_group,unique_values,no_dates,yes_dates
0,civil_rightofway,1340,1320,20
1,civil_structure,3852,36,3816
2,civil_track,20459,20459,0
3,control_systems,13669,5454,8215
4,traction_power_equipment,6012,3002,3010


In [41]:
# Filter DataFrame for 'traction_power_equipment'
traction_power_df = all_floc_with_age[all_floc_with_age['asset_group'] == 'traction_power_equipment']

# Check for rows where all three date columns are missing
# mask = traction_power_df[['construction_year', 'startup_date', 'completion_date']].isnull().all(axis=1)
mask = traction_power_df[['construction_year', 'startup_date']].isnull().all(axis=1)

# Group by 'asset_type' and count items with missing dates
group_counts = traction_power_df.groupby('asset_type').size().reset_index(name='unique_values')
missing_date_counts = traction_power_df[mask].groupby('asset_type').size().reset_index(name='no_dates')

# Merge the two results
result = pd.merge(group_counts, missing_date_counts, on='asset_type', how='outer').fillna(0)

# Ensure the column data types are correct
result['unique_values'] = result['unique_values'].astype(int)
result['no_dates'] = result['no_dates'].astype(int)
result['yes_dates'] = result['unique_values'] - result['no_dates']

result

,asset_type,unique_values,no_dates,yes_dates
0,Autotransformers,249,148,101
1,Auxiliary Supply Transformers,29,15,14
2,Harmonic Filters,35,21,14
3,Isolator,978,530,448
4,Motorised Isolator Motor Units,974,530,444
5,Neutral Sections,86,44,42
6,Overhead Line System,2059,890,1169
7,Section Insulators,436,269,167
8,Traction Power Supply Transformers,421,223,198
9,Traction Power Systems Switchgear,503,259,244


In [23]:
# filtered_df = all_floc_with_age[all_floc_with_age['FLOC_EARTX_TechnicalObjectTypeDescr'] == 'Substation']

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of unique substation floc: {unique_count}")

# filtered_df = all_floc_with_age[
#     (all_floc_with_age['FLOC_EARTX_TechnicalObjectTypeDescr'] == 'Substation') & 
#     (all_floc_with_age['construction_year'].notna() |
#     all_floc_with_age['startup_date'].notna() |
#     all_floc_with_age['completion_date'].notna())
# ]

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of substation floc that has a floc age: {unique_count}")

In [24]:
# filtered_df = all_floc_with_age[all_floc_with_age['asset_group'] == 'civil_rightofway']

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of unique civil rightofway floc: {unique_count}")

# filtered_df = all_floc_with_age[
#     (all_floc_with_age['asset_group'] == 'civil_rightofway') & 
#     (all_floc_with_age['construction_year'].notna() |
#     all_floc_with_age['startup_date'].notna() |
#     all_floc_with_age['completion_date'].notna())
# ]

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of civil rightofway floc that has a floc age: {unique_count}")

In [25]:
# filtered_df = all_floc_with_age[all_floc_with_age['asset_group'] == 'civil_structure']

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of unique structure floc: {unique_count}")

# filtered_df = all_floc_with_age[
#     (all_floc_with_age['asset_group'] == 'civil_structure') & 
#     (all_floc_with_age['construction_year'].notna() |
#     all_floc_with_age['startup_date'].notna() |
#     all_floc_with_age['completion_date'].notna())
# ]

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of structure floc that has a floc age: {unique_count}")

In [26]:
# filtered_df = all_floc_with_age[all_floc_with_age['asset_group'] == 'civil_track']

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of unique civil track floc: {unique_count}")

# filtered_df = all_floc_with_age[
#     (all_floc_with_age['asset_group'] == 'civil_track') & 
#     (all_floc_with_age['construction_year'].notna() |
#     all_floc_with_age['startup_date'].notna() |
#     all_floc_with_age['completion_date'].notna())
# ]

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of civil track floc that has a floc age: {unique_count}")

In [27]:
# filtered_df = all_floc_with_age[all_floc_with_age['asset_group'].isin(['control_systems', 'traction_power_equipment'])]

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of unique electrical floc: {unique_count}")

# filtered_df = all_floc_with_age[
#     (all_floc_with_age['asset_group'].isin(['control_systems', 'traction_power_equipment'])) & 
#     (all_floc_with_age['construction_year'].notna() |
#     all_floc_with_age['startup_date'].notna() |
#     all_floc_with_age['completion_date'].notna())
# ]

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of electrical floc that has a floc age: {unique_count}")

In [28]:
# # Get the file that contain all technical object types (that we care about), linked with maint activities (that we care about), this file were created by Jane
# file_path = r'C:\Users\R893859\Aurizon Operations Limited\Data, Reporting & Analytics - DS08 - NSAP2\Data\Maintenance Policy Modelling\Map Products to Technical Objects\MaintenanceActivityTypes.xlsx'
# maintenance_activity_type_df = pd.read_excel(file_path)

# # Filter that df so it only include the activities we perform periodically (not fix on fail,...)
# filtered_df = maintenance_activity_type_df[maintenance_activity_type_df['Ignore'] != 1]

# # Create a set that contain all the unique technical object types in TechnicalObjects column (All technical object types that Jane filled in)
# unique_technical_objects = set()
# for items in filtered_df['TechnicalObjects']:
#     unique_technical_objects.update(item.strip() for item in items.split(', '))

# unique_count = all_floc_with_age[all_floc_with_age['FLOC_EARTX_TechnicalObjectTypeDescr'].isin(unique_technical_objects)]['FLOC_STRNO_FunctionalLocation'].nunique()

# print(f"Number of unique floc with TechnicalObjectTypeDescr: {unique_count}")

# # Filter the DataFrame to include only rows where FLOC_EARTX_TechnicalObjectTypeDescr is the ones that we care about (unique_technical_objects), created above
# filtered_df = all_floc_with_age[all_floc_with_age['FLOC_EARTX_TechnicalObjectTypeDescr'].isin(unique_technical_objects)]

# # Further filter to include rows where at least one of the datetime columns is not null
# filtered_df = filtered_df[
#     filtered_df['construction_year'].notna() | 
#     filtered_df['startup_date'].notna() | 
#     filtered_df['completion_date'].notna()
# ]

# # Count the number of unique values in the FLOC_STRNO_FunctionalLocation column
# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()

# print(f"Number of unique floc with TechnicalObjectTypeDescr (that we care about) AND has a floc_age: {unique_count}")

In [29]:
# filtered_df = all_floc_with_age[all_floc_with_age['asset_group'].notna()]

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of unique floc that were identified by Mike: {unique_count}")

# filtered_df = all_floc_with_age[
#     (all_floc_with_age['asset_group'].notna()) & 
#     (all_floc_with_age['construction_year'].notna() |
#     all_floc_with_age['startup_date'].notna() |
#     all_floc_with_age['completion_date'].notna())
# ]

# unique_count = filtered_df['FLOC_STRNO_FunctionalLocation'].nunique()
# print(f"Number of unique floc that were identified by Mike AND has a floc age: {unique_count}")

In [30]:
# duplicate_rows = all_floc_with_age[all_floc_with_age.duplicated()]

# duplicate_rows.sort_values(by='FLOC_STRNO_FunctionalLocation')

In [33]:
all_floc_with_age[all_floc_with_age['FLOC_EARTX_TechnicalObjectTypeDescr'] == 'Substation']

,FLOC_STRNO_FunctionalLocation,FLOC_PLTXT_FunctionalLocationDescr,FLOC_EARTX_TechnicalObjectTypeDescr,FLOC_STORT_LocationDescr,asset_type,asset_group,construction_year,startup_date
60432,ST000680-13,Oonooie FS,Substation,Oonooie,NaN,NaN,NaT,NaT
60433,ST000700-13,Wandoo FS,Substation,Wandoo,NaN,NaN,NaT,NaT
60434,ST000710-18,Coppabella FS,Substation,Coppabella,NaN,NaN,NaT,NaT
60435,ST000780-09,Moranbah South TSC,Substation,Moranbah South,NaN,NaN,NaT,NaT
60436,ST000800-08,Mount McLaren FS,Substation,Mount McLaren,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...
188243,ST009070-18,EP MI North PSC,Substation,Epala,NaN,NaN,NaT,NaT
192062,ST009460-15,YN AT PSC,Substation,Yarwun,NaN,NaN,NaT,NaT
192063,ST007820-12,GR AT PSC,Substation,Gracemere,NaN,NaN,NaT,NaT
195650,ST006025,Olive Downs 27.808km PSE,Substation,Olive Downs,NaN,NaN,NaT,NaT


In [42]:
all_floc_with_age.to_excel(r'C:\Users\R893859\Aurizon Operations Limited\Data, Reporting & Analytics - DS08 - NSAP2\Data\Floc Age\floc_age_without_FAR.xlsx', index=False)

In [ ]:
# C:\Users\R893859\Aurizon Operations Limited\Data, Reporting & Analytics - DS08 - NSAP2\Data\Floc Age